# The notebook for running DSSP on selected wild-mutation pairs. 

In [1]:
from Bio.PDB import DSSP
from Bio.PDB import PDBParser
import warnings
from Bio import BiopythonWarning
import pandas as pd
from Bio.PDB import PDBIO
from Bio.PDB import Dice
import numpy as np
import Bio
import sys

Notice that in this demo we are using the windows version of dssp (.exe). You may use the dssp on other platform as long as you change the envriment settings. Also you need to make sure you have all the pdb files downloaded. 

In [2]:
residue_max_acc = { 'A': 106.0, 'R': 248.0, 'N': 157.0,
                   'D': 163.0, 'C': 135.0, 'Q': 198.0, 
                   'E': 194.0, 'G': 84.0, 'H': 184.0, 
                   'I': 169.0, 'L': 164.0, 'K': 205.0, 
                   'M': 188.0, 'F': 197.0, 'P': 136.0, 
                   'S': 130.0, 'T': 142.0, 'W': 227.0, 
                   'Y': 222.0, 'V': 142.0, 'X':169.6 } 

class Chain_filter:
    def __init__(self, chain_id):
        self.chain_id = chain_id
    ## accept all model
    def accept_model(self, model):
        return 1
    ## accept the chains in the group
    def accept_chain(self, chain): 
        if chain.get_id() in self.chain_id: 
            return 1
        return 0
    # accept all residue in the chain
    def accept_residue(self, residue): 
        return 1
    # accept all atoms
    def accept_atom(self, atom):
        return 1
    
def extract_chain2pdb(structure, chain_ids, filename):
    sel = Chain_filter(chain_ids) 
    pdbio = PDBIO() 
    pdbio.set_structure(structure) 
    pdbio.save(filename, sel) 

def calculate_sasa(directory,select_mutation_num=None,select_chain_num=None):
    """
    Read in the csv file with a list of wildtypes and mutations.
    Calculate the a pair of Delta_SASA for each protei; also calculate the Delta Delta_SASA (wt - mut)
    return a dataframe containing the information
    """
    warnings.simplefilter('ignore',BiopythonWarning)
    parser = PDBParser()
    result = []
    
    # read in the pairs file
    # notice that chains are the same for each pairs of the protein inside the file
    wt_mu_pair = pd.read_csv(directory,index_col=0)
    
    if select_mutation_num!=None:
        wt_mu_pair = wt_mu_pair.loc[wt_mu_pair.mutation_num == select_mutation_num,:]
    
    if select_chain_num!=None:
        wt_mu_pair = wt_mu_pair.loc[wt_mu_pair.chain_num == select_chain_num,:]
        
    track = 0
    total = wt_mu_pair.shape[0]
    
    for keys, vals in wt_mu_pair.iterrows():
        # obtian the directory to the wildtype/mutant files
        wt_name = vals['wildtype']
        mu_name = vals['mutant']
        
        # specify the pdb files directory here
        wt_file = './dssp/pdb/pdb' + wt_name+'.ent'
        mu_file = './dssp/pdb/pdb' + mu_name+'.ent'
        
        
        # get the structure of complex
        wt_struc = parser.get_structure(wt_name, wt_file)
        mu_struc = parser.get_structure(mu_name, mu_file)
        
        try:
            # find the dssp result of the complex
            wt_dssp = DSSP(wt_struc[0], wt_file, 'dssp.exe')
            # in case there are failure
            while len(wt_dssp)==0:
                wt_dssp = DSSP(wt_struc[0], wt_file, 'dssp.exe')

            mu_dssp = DSSP(mu_struc[0], mu_file, 'dssp.exe')
            # in case there are failure
            while len(mu_dssp)==0:
                mu_dssp = DSSP(mu_struc[0], mu_file, 'dssp.exe')
        except:
            print("The following pair has corrupted pdb:", wt_name, mu_name)
            continue
            
        # find the total sasa of the complex
        Total_SASA_wt = np.sum([item[3]*residue_max_acc[item[1]] for item in wt_dssp if item[1]!='X'])
        Total_SASA_mu = np.sum([item[3]*residue_max_acc[item[1]] for item in mu_dssp if item[1]!='X'])
        
        # get the number of chains and divide them into groups
        chain_count = len(wt_struc[0])
        first_group = int(chain_count/2)
        index = 0
        group1 = []
        group2 = []
        for chain in wt_struc[0]:
            if index < first_group:
                group1.append(chain.get_id())
            else:
                group2.append(chain.get_id())
            index += 1
    
        Chain_in_g1 = ''.join(group1) 
        Chain_in_g2 = ''.join(group2)
        
            
        if len(group2) == 0 or len(group1)==0 or chain_count <= 1:
            continue 
        
        try:
            ## wild type
            # filter to group 1
            wt_g1_struc = parser.get_structure(wt_name, wt_file)
            extract_chain2pdb(wt_g1_struc,group1, 'temp.ent')
            wt_g1_dssp = DSSP(wt_g1_struc[0],'temp.ent','dssp.exe')
            while len(wt_g1_dssp)==0:
                wt_g1_dssp = DSSP(wt_g1_struc[0], 'temp.ent', 'dssp.exe')

            SASA_wt_g1 = np.sum([item[3]*residue_max_acc[item[1]] for item in wt_g1_dssp if item[1]!='X'])

            # filter to group 2
            wt_g2_struc = parser.get_structure(wt_name, wt_file)
            extract_chain2pdb(wt_g2_struc,group2, 'temp.ent')
            wt_g2_dssp = DSSP(wt_g2_struc[0],'temp.ent','dssp.exe')
            while len(wt_g2_dssp)==0:
                wt_g2_dssp = DSSP(wt_g2_struc[0], 'temp.ent', 'dssp.exe')

            SASA_wt_g2 = np.sum([item[3]*residue_max_acc[item[1]] for item in wt_g2_dssp if item[1]!='X'])

            ## mutant
            # filter to group 1
            mu_g1_struc = parser.get_structure(mu_name, mu_file)
            extract_chain2pdb(mu_g1_struc,group1, 'temp.ent')
            mu_g1_dssp = DSSP(mu_g1_struc[0],'temp.ent','dssp.exe')
            while len(mu_g1_dssp)==0:
                mu_g1_dssp = DSSP(mu_g1_struc[0], 'temp.ent', 'dssp.exe')

            SASA_mu_g1 = np.sum([item[3]*residue_max_acc[item[1]] for item in mu_g1_dssp if item[1]!='X'])

            # filter to group 2
            mu_g2_struc = parser.get_structure(mu_name, mu_file)
            extract_chain2pdb(mu_g2_struc,group2, 'temp.ent')
            mu_g2_dssp = DSSP(mu_g2_struc[0],'temp.ent','dssp.exe')
            while len(mu_g2_dssp)==0:
                mu_g2_dssp = DSSP(mu_g2_struc[0], 'temp.ent', 'dssp.exe')

            SASA_mu_g2 = np.sum([item[3]*residue_max_acc[item[1]] for item in mu_g2_dssp if item[1]!='X'])
        except:
            SASA_wt_g1 = np.nan
            SASA_wt_g2 = np.nan
            SASA_mu_g1 = np.nan
            SASA_mu_g2 = np.nan
            print("The following pair cause error in DSSP:", wt_name, mu_name)
        
        result.append( (wt_name,mu_name,
                        Total_SASA_wt,SASA_wt_g1,SASA_wt_g2,
                        Total_SASA_mu,SASA_mu_g1,SASA_mu_g2,
                        Chain_in_g1,Chain_in_g2))
        track += 1
        sys.stdout.write("\r"+str(track)+'/'+str(total)+" done.")
        sys.stdout.flush()
    return result

### In this demo, we use the constructed wild-mutant pairs with 1,2 or 3 mutations from PDB. Please refer to our paper for how we use sequence alignment to generate the wild-mutant pairs. 

In [3]:
res = calculate_sasa('./dssp/wt_mu_pairs.csv',select_mutation_num=1,select_chain_num=2)

AttributeError: 'DataFrame' object has no attribute 'mutation_num'

In [ ]:
df = pd.DataFrame(res, columns=['wt_name','mu_name',
                        'Total_SASA_wt','SASA_wt_g1','SASA_wt_g2',
                        'Total_SASA_mu','SASA_mu_g1','SASA_mu_g2',
                        'Chain_in_G1','Chain_in_G2'])
df.to_csv('./dssp/dssp_sasa.csv')